## Introduction to basket modelling with SHOPPER
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/artefactory/choice-learn/blob/main/notebooks/basket_models/shopper_tutorial.ipynb)

We use a synthetic dataset to demonstrate how to use the SHOPPER model [1].

In [ ]:
# Install necessary requirements

# If you run this notebook on Google Colab, or in standalone mode, you need to install the required packages.
# Uncomment the following lines:

# !pip install choice-learn

# If you run the notebook within the GitHub repository, you need to run the following lines, that can skipped otherwise:
import sys

sys.path.append("../../")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf

from choice_learn.basket_models import Trip, TripDataset
from choice_learn.basket_models import Shopper

In [ ]:
# List all physical GPUs
physical_gpus = tf.config.list_physical_devices("GPU")
print(f"Available physical GPUs: {physical_gpus}")
# Select GPUs to use
selected_gpus = []  # Choose the GPUs you want to use ([] = CPU)
# Set the selected GPUs to be visible
tf.config.set_visible_devices(selected_gpus, "GPU")
# Verify the visible GPUs
visible_gpus = tf.config.get_visible_devices("GPU")
print(f"Visible GPUs: {visible_gpus}")

# Limit GPU memory growth
if physical_gpus:
  try:
    for gpu in physical_gpus:
      # Allocate only as much GPU memory as needed for the runtime allocations
      tf.config.experimental.set_memory_growth(gpu, True)
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

### Dataset

Let's consider a simple dataset where we have only six items sold in two different stores:
- The first store sells items [0, 1, 2, 3, 4] and has observed baskets [1, 0], [2, 0], [1, 3, 4, 0];
- The second store sells items [0, 1, 5, 6] and has observed baskets [1, 0], [6, 5, 0];

with 0 the checkout item.

Let's say that each basket has been seen 100 times.

In [ ]:
n_items = 7
assortment_store_1 = np.array([1, 1, 1, 1, 1, 0, 0])
assortment_store_2 = np.array([1, 1, 0, 0, 0, 1, 1])

available_items = np.array([assortment_store_1, assortment_store_2])

In [ ]:
print(f"The list of available items are encoded as availability matrices indicating the availability (1) or not (0) of the products:\n{available_items=}\n")
print(
    "Here, the variable 'available_items' can be read as:\n",
    f"- Assortment 1 = {[i for i in range(n_items) if assortment_store_1[i]==1]}\n",
    f"- Assortment 2 = {[i for i in range(n_items) if assortment_store_2[i]==1]}"
)

In [ ]:
num_baskets = 100

purchases_stores_1 =[[1, 0], [2, 0], [1, 3, 4, 0]]
purchases_stores_2 = [[1, 0], [6, 5, 0]]

Now that we have our synthetic data, let's transform it as a TripDataset that will be fed to the model.

In [ ]:
# First we create a list of Trip objects:
trips_list = []

for i in range(num_baskets):
    trip = Trip(
        purchases=purchases_stores_1[0],
        # Let's consider here totally random prices for the products
        prices=np.random.uniform(1, 10, n_items),
        assortment=0
    )
    trips_list.append(trip)

    trip = Trip(
        purchases=purchases_stores_1[1],
        prices=np.random.uniform(1, 10, n_items),
        assortment=0
    )
    trips_list.append(trip)

    trip = Trip(
        purchases=purchases_stores_1[2],
        prices=np.random.uniform(1, 10, n_items),
        assortment=0
    )
    trips_list.append(trip)

    trip = Trip(
        purchases=purchases_stores_2[0],
        prices=np.random.uniform(1, 10, n_items),
        assortment=1
    )
    trips_list.append(trip)

    trip = Trip(
        purchases=purchases_stores_2[1],
        prices=np.random.uniform(1, 10, n_items),
        assortment=1
    )
    trips_list.append(trip)

dataset = TripDataset(trips=trips_list, available_items=available_items)

### Training SHOPPER model

Now we can fit a SHOPPER model.

In [ ]:
# Hyperparameters

# Preferences and price effects are represented by latent variables of size 4 and 3, respectively.
latent_sizes = {"preferences": 4, "price": 3}
# We use 1 negative sample for each positive sample during the training phase.
n_negative_samples = 1
optimizer = "adam"
lr = 1e-3
epochs = 100
batch_size = 256

In [ ]:
# Model: items fixed effect + items interactions + price effects
shopper = Shopper(
    item_intercept=False,
    price_effects=True,
    seasonal_effects=False,
    latent_sizes=latent_sizes,
    n_negative_samples=n_negative_samples,
    optimizer=optimizer,
    lr=lr,
    epochs=epochs,
    batch_size=batch_size,
)
# Feel free to explore other models by changing the hyperparameters!

The SHOPPER model can integrate store effects as well as seasonality. Check the documentation if you want to know more about it.

In [ ]:
# Instantiate the model
shopper.instantiate(n_items=n_items)

# Train the model
history = shopper.fit(trip_dataset=dataset)

In [ ]:
plt.plot(history["train_loss"])
plt.xlabel("Epoch")
plt.ylabel("Training Loss")
plt.legend()
plt.title("Training of SHOPPER model")
plt.show()

### Inference with SHOPPER model

We evaluate the model on the validation dataset.

In [ ]:
n_permutations = 2

# You can choose how many basket permuations are used to evaluate the model
nll = shopper.evaluate(dataset, n_permutations)

In [ ]:
print(f"Mean negative log-likelihood on the dataset: {nll:.4f}.")

print("\nWe can see that the more complex the model, the lower the negative log-likelihood.")

We can also compute various utilities and probabilities.

In [ ]:
# Item utilities
item_batch_inference=np.array([2, 0, 4])
basket_inference = np.array([1, 3])
price_inference = 5.
available_items_inference = np.ones(dataset.n_items)
available_items_inference[4] = 0  # Consider that item 4 is not available during inference
assortment_inference = np.array(
    [
        item_id for item_id in dataset.get_all_items() if available_items_inference[item_id] == 1
    ]
)

item_utilities = shopper.compute_batch_utility(
    item_batch=item_batch_inference,
    basket_batch=np.tile(basket_inference, (3, 1)),
    store_batch=np.array([0]), # 0 if not defined
    week_batch=np.array([0]), # 0 if not defined
    price_batch=np.tile(price_inference, 3),
    available_item_batch=np.tile(available_items_inference, (3, 1)),
)

print(
    f"Considering the assortment (ie the set of available items) {assortment_inference} with prices {price_inference},",
    f"and a basket with the items {basket_inference}.\n",
    f"Under these circumstances, the utility of the selected items are:"
)
for i, item_id in enumerate(item_batch_inference):
    if item_id == 0:
        print(f"    - Item {item_id} (checkout item): {item_utilities[i]:.4f}")
    else:
        print(f"    - Item {item_id}: {item_utilities[i]:.4f}")

In [ ]:
# Item likelihoods
item_batch=np.array([2, 0, 4])
item_likelihoods = shopper.compute_item_likelihood(
    basket=basket_inference,
    available_items=available_items_inference,  # Consider all items available
    store=0,  # 0 if not defined
    week=0,  # 0 if not defined
    prices=price_inference,
)

print(
    f"Considering the assortment (ie the set of available items) {assortment_inference} with prices {price_inference},",
    f"and a basket with the items {basket_inference}.\n",
    f"Under these circumstances, the likelihoods that each item will be the next item added to the basket are:"
)
for i, item_id in enumerate(dataset.get_all_items()):
    if item_id == 0:
        print(f"    - Item {item_id} (checkout item, the customer decides to end his shopping trip): {item_likelihoods[i]:.4f}")
    else:
        print(f"    - Item {item_id}: {item_likelihoods[i]:.4f}")
print(f"\nN.B.: The item likelihoods sum to {np.sum(item_likelihoods):.4f}.")

In [ ]:
# Ordered basket likelihood
basket_ordered = np.array([1, 3, 0])
basket_ordered_likelihood = shopper.compute_ordered_basket_likelihood(
    basket=basket_ordered,
    available_items=available_items_inference,  # Consider all items available
    store=0,
    week=0,
    prices=price_inference,
)

print(f"Likelihood for ordered basket {basket_ordered}: {basket_ordered_likelihood:.4f}.")

In [ ]:
# (Unordered) basket likelihood
n_permutations = 2

basket = np.array([1, 3, 0])
basket_likelihood = shopper.compute_basket_likelihood(
    basket=basket,
    available_items=available_items_inference,  # Consider all items available
    store=0,
    week=0,
    prices=price_inference,
    n_permutations=n_permutations,
)
print(f"Likelihood for (unordered) basket {basket}: {basket_likelihood:.4f} (with {n_permutations} permutations to approximate all possible orders).")

### References
[1] SHOPPER: A Probabilistic Model of Consumer Choice with Substitutes and Complements, Ruiz, F. J. R.; Athey, S.; Blei, D. M. (2019), Annals of Applied Statistic
(URL: https://arxiv.org/abs/1711.03560)